# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [132]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint
# import gmaps
import gmaps

# Import API key
from config.api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [134]:
df_base = pd.read_csv('output/cities.csv', index_col=0)
df_base

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Anadyr,90,RU,1587162143,78,64.75,177.48,12.20,13.42
1,Carnarvon,89,AU,1587162149,88,-24.87,113.63,75.20,5.82
2,Saldanha,70,ZA,1587162150,72,-33.01,17.94,59.00,9.17
3,Codrington,100,AU,1587162150,77,-38.27,141.97,54.73,9.53
4,Rikitea,1,PF,1587161996,65,-23.12,-134.97,77.18,2.51
...,...,...,...,...,...,...,...,...,...
561,Komsomolets,98,KZ,1587162232,67,53.75,62.07,46.29,15.70
562,Rio Grande,4,BR,1587162037,83,-32.03,-52.10,62.01,10.45
563,Plérin,66,FR,1587162232,87,48.54,-2.78,55.00,1.12
564,Abu Samrah,9,SY,1587162232,43,35.30,37.18,64.74,5.91


In [135]:
locations = df_base[['Lat', 'Lng']].astype(float)
humidity = df_base['Humidity'].astype(float)

In [141]:

fig = gmaps.figure()

heatLayer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=10 ,
    point_radius=1
)

fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
df_trimed = df_base.loc[(df_base['Max Temp'] <= 65) & (df_base['Max Temp'] >= 45) & (df_base['Humidity'] <= 40)].reset_index(drop=True).copy()

In [65]:
hotel_df = df_trimed.copy()
hotel_df['Hotel Name'] = ''
hotel_names = []

In [66]:
for i, row in hotel_df.iterrows():

    # geocoordinates
    target_coordinates = str(row['Lat']) + ',' +  str(row['Lng'])
    target_search = "Hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    tempor = response.json()
    try:
        hotel_names.append(tempor['results'][0]['name'])
        print(tempor['results'][0]['name'])
    except (KeyError, IndexError):
        print("Could not find Hotel")
        hotel_names.append('N/A')
print(hotel_names)

Los Cauquenes Resort & Spa
Could not find Hotel
El Rancho Hotel
Quality Inn
Haoli Business Hotel
Kempinski Hotel Yinchuan
Best Western Danbury/Bethel
Zhangjiakou International Hotel
Mandal Hotel
The Trainmaster's Inn
Canopy by Hilton Minneapolis Mill District
Blue Poppy Resorts
Could not find Hotel
Country Inn & Suites by Radisson, Gillette, WY
Comfort Inn & Suites Lovington
Quality Inn Payson I-15
Casa Grande de Anllo
Towo Topping Hotel
Could not find Hotel
Holiday Inn Express & Suites Klamath Falls Central
['Los Cauquenes Resort & Spa', 'N/A', 'El Rancho Hotel', 'Quality Inn', 'Haoli Business Hotel', 'Kempinski Hotel Yinchuan', 'Best Western Danbury/Bethel', 'Zhangjiakou International Hotel', 'Mandal Hotel', "The Trainmaster's Inn", 'Canopy by Hilton Minneapolis Mill District', 'Blue Poppy Resorts', 'N/A', 'Country Inn & Suites by Radisson, Gillette, WY', 'Comfort Inn & Suites Lovington', 'Quality Inn Payson I-15', 'Casa Grande de Anllo', 'Towo Topping Hotel', 'N/A', 'Holiday Inn Exp

In [75]:
hotel_df['Hotel Name'] = hotel_names

In [76]:
hotel_df.loc[hotel_df['Hotel Name'] == 'N/A']

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,Albany,100,US,1587162153,32,42.60,-73.97,46.99,5.01,N/A
12,Faya,0,SA,1587162208,34,18.39,42.45,64.40,8.05,N/A
18,Gazli,4,UZ,1587162219,28,40.13,63.45,64.45,2.95,N/A


In [79]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != 'N/A'].reset_index(drop=True)

In [86]:
mark_lats = hotel_df['Lat'].to_list()
mark_lngs = hotel_df['Lng'].to_list()
marker_coords = []
for x in range(len(mark_lats)):
    marker_coords.append([mark_lats[x] , mark_lngs[x]])


In [121]:
names = hotel_df['Hotel Name'].to_list()
names_fixed = []
for x in names:
    names_fixed.append(x.replace('&', 'and'))

In [128]:
marker_names = []
a = hotel_df['City'].to_list()
b = hotel_df['Country'].to_list()
for x in range(len(a)):
    string = f'{a[x]} , {b[x]} , {names_fixed[x]}'
    marker_names.append(string)
marker_names

['Ushuaia , AR , Los Cauquenes Resort and Spa',
 'Gallup , US , El Rancho Hotel',
 'Raton , US , Quality Inn',
 'Kumul , CN , Haoli Business Hotel',
 'Yinchuan , CN , Kempinski Hotel Yinchuan',
 'Bethel , US , Best Western Danbury/Bethel',
 'Zhangjiakou , CN , Zhangjiakou International Hotel',
 'Mandalgovi , MN , Mandal Hotel',
 "Palmer , US , The Trainmaster's Inn",
 'Saint Anthony , US , Canopy by Hilton Minneapolis Mill District',
 'Joshīmath , IN , Blue Poppy Resorts',
 'Gillette , US , Country Inn and Suites by Radisson, Gillette, WY',
 'Lovington , US , Comfort Inn and Suites Lovington',
 'Payson , US , Quality Inn Payson I-15',
 'Bereda , ES , Casa Grande de Anllo',
 'Pingliang , CN , Towo Topping Hotel',
 'Klamath Falls , US , Holiday Inn Express and Suites Klamath Falls Central']

In [137]:
marker_layer = gmaps.marker_layer( 
    locations=marker_coords,
    label=marker_names
)

In [138]:
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [103]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]